In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab_Notebooks/DL-Sigma2021/Deep Learning Project_SIGMA M2_2021"

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/DL-Sigma2021/Deep Learning Project_SIGMA M2_2021


In [3]:
import numpy as np
import os, shutil
import librosa
import soundfile as sf
import time

In [4]:
def read_audio(filepath, sample_rate, normalize=True):
    """Read an audio file and return it as a numpy array"""
    audio, sr = librosa.load(filepath, sr=sample_rate)
    if normalize:
      div_fac = 1 / np.max(np.abs(audio)) / 3.0
      audio = audio * div_fac
    return audio, sr

def add_noise_to_clean_audio(clean_audio, noise_signal):
    """Adds noise to an audio sample"""
    if len(clean_audio) >= len(noise_signal):
        # print("The noisy signal is smaller than the clean audio input. Duplicating the noise.")
        while len(clean_audio) >= len(noise_signal):
            noise_signal = np.append(noise_signal, noise_signal)

    ## Extract a noise segment from a random location in the noise file
    ind = np.random.randint(0, noise_signal.size - clean_audio.size)

    noiseSegment = noise_signal[ind: ind + clean_audio.size]

    speech_power = np.sum(clean_audio ** 2)
    noise_power = np.sum(noiseSegment ** 2)
    noisyAudio = clean_audio + np.sqrt(speech_power / noise_power) * noiseSegment
    return noisyAudio

In [5]:
fs = 16e3
Noise_Path = './Audio_Result'
final_path = './dataset/speech_TRAIN_1/noisy_TRAIN_1/'
final_clean_path = './dataset/speech_TRAIN_1/clean_TRAIN_1/'
final_path2 = './dataset/speech_TRAIN_2/noisy_TRAIN_2/'
final_path3 = './dataset/speech_TEST/clean_TEST/'
path = './dataset/speech_TRAIN_1/TIMIT_TRAIN_1/'

files = []
name = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.WAV' in file:
            files.append(os.path.join(r, file))
            name.append(file)

for f in range(len(files)):
    print('Train_1 in Progress: ',f,"/",len(files),end = "\r")
    cleanAudio, sr = read_audio(files[f], sample_rate=fs)
    time.sleep(0.1)
    sf.write(final_clean_path+name[f], cleanAudio, int(sr))
path2 = './dataset/speech_TRAIN_2/TIMIT_TRAIN_2/'

files2 = []
name2 = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path2):
    for file in f:
        if '.WAV' in file:
            files2.append(os.path.join(r, file))
            name2.append(file)
            
path3 = './dataset/speech_TEST/TIMIT_TEST/'

files3 = []
name3 = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path3):
    for file in f:
        if '.WAV' in file:
            files3.append(os.path.join(r, file))
            name3.append(file)
for f in range(len(files)):
    cleanAudio, sr = read_audio(files[f], sample_rate=fs)
    noiseAudio, sr = read_audio(os.path.join(Noise_Path, 'babble.wav'), sample_rate=fs)
    noisyAudio = add_noise_to_clean_audio(cleanAudio, noiseAudio)
    sf.write(final_path+name[f], noisyAudio, int(sr))
noiseAudio, sr = read_audio(os.path.join(Noise_Path, 'babble.wav'), sample_rate=fs)
for f in range(len(files2)):
    print('Train_2 in Progress: ',f,"/",len(files2),end = "\r")
    cleanAudio, sr = read_audio(files2[f], sample_rate=fs)
    noisyAudio = add_noise_to_clean_audio(cleanAudio, noiseAudio)
    sf.write(final_path2+name2[f], noisyAudio, int(sr))
    time.sleep(0.1)
for f in range(0,len(files3)):
    print('Test in Progress: ',f,"/",len(files3),end = "\r")
    cleanAudio, sr = read_audio(files3[f], sample_rate=fs)
    time.sleep(0.1)
    #noiseAudio, sr = read_audio(os.path.join(Noise_Path, 'babble.wav'), sample_rate=fs)
    #noisyAudio = add_noise_to_clean_audio(cleanAudio, noiseAudio)
    sf.write(final_path3+name3[f], cleanAudio, int(sr))